<a href="https://colab.research.google.com/github/asrithap07/ChildrensBookGenerator/blob/main/TrainingDrSeussModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation using GPT (Using Huggingface)

## Project Setup

In [ ]:
# Maybe use this:



import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip uninstall transformers
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
import torch
import shutil
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments, GPTNeoForCausalLM, GPT2Tokenizer


from google.colab import drive


## Data Preparation

In [ ]:
# Load data into colab
#!wget https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt

In [ ]:
# Connects colab to google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#shutil.copy("/content/shakespeare.txt","drive/MyDrive/AICamp/nlp")


In [ ]:
# Read the text file and returns list of lines in text
def read_file(file_path):
    with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
    return lines


In [ ]:
# @title
file_path = "drive/MyDrive/AICamp/nlp/Datasets/dr-seuss-with-no-titles.txt"



new_text = []


texts = read_file(file_path)


line_count = 0

paragraphs = []
paragraph = []
for text in texts:
  if line_count < 8:
    line_count += 1
    paragraph.append(text)
  else:
    paragraphs.append(''.join(paragraph))
    line_count = 0
    paragraph = []

print(paragraphs)
# Remove unnecessary texts


['The sun did not shine.\nIt was too wet to play.\nSo we sat in the house\nAll that cold cold wet day.\nI sat there with Sally.\nWe sat there we two.\nAnd I said How I wish\nWe had something to do!\n', 'And too cold to play ball.\nSo we sat in the house.\nWe did nothing at all.\nSo all we could do was to\nSit!\nSit!\nSit!\nSit!\n', 'Not one little bit.\nBUMP!\nAnd then\nsomething went BUMP!\nHow that bump made us jump!\nWe looked!\nThen we saw him step in on the mat!\nWe looked!\n', 'The Cat in the Hat!\nAnd he said to us\nWhy do you sit there like that?\nI know it is wet\nAnd the sun is not sunny.\nBut we can have\nLots of good fun that is funny!\nI know some good games we could play\n', 'I know some new tricks\nSaid the Cat in the Hat.\nA lot of good tricks.\nI will show them to you.\nYour mother\nWill not mind at all if I do.\nThen Sally and I\nDid not know what to say.\n', 'For the day.\nBut our fish said No! No!\nMake that cat go away!\nTell that Cat in the Hat\nYou do NOT want to

In [ ]:
# Prepare paragraphs
datas = paragraphs
print(len(datas))
for data in datas:
  if len(data)<1:
    datas.remove(data)
print(len(datas))

111
111


In [ ]:
# Custome dataset class to load dataset
class DrSeussDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>'
                                        + txt +
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length,
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

## Initialize tokenizer, model

In [ ]:
# Set the random seed to a fixed value to get reproducible results
torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()

# Resize the token embeddings because we've just added 3 new tokens
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

## Train/Test Split data

In [ ]:
max_length = max([len(tokenizer.encode(paragraph)) for paragraph in datas])

# Load dataset
dataset = DrSeussDataset(paragraphs, tokenizer, max_length)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

max_length

119

In [ ]:
tokenizer.batch_decode(val_data[0])

['<|startoftext|> Gooey goo for chewy chewing!\nThats what that Goo Goose is doing.\nDo you choose to chew goo too sir?\nIf sir you sir choose to chew sir\nwith the Goo Goose chew sir.\nDo sir.\nMr. Fox sir\nI wont do it.\n<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>',
 '""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!']

## Train Model

In [ ]:
# # Here I will pass the output directory where
# # the model predictions and checkpoints will be stored,
# # batch sizes for the training and validation steps,
# # and warmup_steps to gradually increase the learning rate
# learning_rates = [5e-5, 3e-5, 1e-5]

# for learning_rate in learning_rates:

#     training_args = TrainingArguments(output_dir=f'./results_{learning_rate}',
#                                       num_train_epochs=5,
#                                       logging_steps=1000,
#                                       save_steps=1000,
#                                       evaluation_strategy='steps',
#                                       eval_steps=1000,
#                                       per_device_train_batch_size=2,
#                                       per_device_eval_batch_size=2,
#                                       warmup_steps=100,
#                                       learning_rate=learning_rate,
#                                       weight_decay=0.01,
#                                       logging_dir=f'./logs_{learning_rate}')

#     trainer = Trainer(model=model, args=training_args,
#                       train_dataset=train_data,
#                       eval_dataset=val_data,
#                       # This custom collate function is necessary
#                       # to built batches of data
#                       data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
#                   'attention_mask': torch.stack([f[1] for f in data]),
#                   'labels': torch.stack([f[0] for f in data])})

#     # Start training process!
#     print(f"Training result for learning rate: {learning_rate}")
#     trainer.train()
#     print("\n\n")

BAsed on the results above, it looks like model trained with learning rate = 5e-5 is more promising than others.

In [ ]:
training_args = TrainingArguments(output_dir=f'./results',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,
                                      logging_dir=f'./logs')

trainer = Trainer(model=model, args=training_args,
                  train_dataset=train_data,
                  eval_dataset=val_data,
                  # This custom collate function is necessary
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=1.7092474365234376, metrics={'train_runtime': 29.7971, 'train_samples_per_second': 16.612, 'train_steps_per_second': 8.39, 'total_flos': 30051570447360.0, 'train_loss': 1.7092474365234376, 'epoch': 5.0})

In [ ]:
# Save model in the specified file path
trainer.save_model("drive/MyDrive/AICamp/nlp/models/TestingParameters")

In [ ]:
tokenizer.save_pretrained("drive/MyDrive/AICamp/nlp/models/TestingParameters")

('drive/MyDrive/AICamp/nlp/models/TestingParameters/tokenizer_config.json',
 'drive/MyDrive/AICamp/nlp/models/TestingParameters/special_tokens_map.json',
 'drive/MyDrive/AICamp/nlp/models/TestingParameters/vocab.json',
 'drive/MyDrive/AICamp/nlp/models/TestingParameters/merges.txt',
 'drive/MyDrive/AICamp/nlp/models/TestingParameters/added_tokens.json')

## Checking Model Output

In [ ]:
input_text = "Once upon a time,"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids.cuda(), do_sample=True, max_length=1000, temperature=0.7, num_return_sequences=1)

generated_story = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Story:")
print(generated_story)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Story:
Once upon a time, there was no such thing as a good game.
So when the ball came down the chimney with a thunk
And the rain was falling in torrents all about
Then the fish said to the cat:
The cat is not in a box.
He is in a box.



In [ ]:
# ! transformers-cli env

## Upload model to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
# Create your repo first to upload the model
api.create_repo(repo_id="gpt2-dr-seuss-generators")

ValueError: ignored

In [ ]:
# Upload your model to huggingface. You can clone the repo anytime to use the model.
import os

model_pth = "drive/MyDrive/AICamp/nlp/models/Dr Seuss"

files = os.listdir(model_pth)
for fi in files:
    print(os.path.join(model_pth, fi))

    api.upload_file(
        path_or_fileobj=os.path.join(model_pth, fi),
        path_in_repo=fi,
        repo_id="dpseigel/gpt2-dr-seuss-generators",
        repo_type="model",
    )